# Load Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.optimizers import RMSprop, SGD
import os


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: {}, and its: {}".format(len(physical_devices), physical_devices) )
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available: 1, and its: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load Data

In [3]:
def load_data(path):
    #1. load
    data = pd.read_csv(path)
    
    #2. preprocessing
    X = data.drop(columns=['label'])
    y = data['label'].values
    
    # robust scaler
    scalerRobust = RobustScaler()
    X = scalerRobust.fit_transform(X)
    
    # Encode to int label
    labelencoder = LabelEncoder()
    y = labelencoder.fit_transform(y)
    # To categorical Label
    y = to_categorical(y)
    
    return X, y 

In [4]:
X, y = load_data('./13-01-2022_05-47-39_feature.csv')

# Build KFold model

In [5]:
def create_model():
    model = Sequential()
    model.add(Dense(128, activation='sigmoid', input_dim=X.shape[1], name="hidden_layer_1"))
    model.add(Dense(256, activation='sigmoid', name="hidden_layer_2"))
    model.add(Dense(y.shape[1], activation='softmax', name='output'))
    
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = adam,
    metrics = ['accuracy']
    )
    
    return model

In [6]:
avg_result = []

In [7]:
def kfold(n_split, X, y):
    
    kFold = KFold(n_splits=n_split, shuffle=True, random_state=42)
    
    for train_index, test_index in kFold.split(X):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

        model = create_model()
        model.fit(X_train, y_train, batch_size=64, epochs=500, verbose=0)
        evaluated = model.evaluate(X_test, y_test)
        print('Model evaluation ', evaluated)
        
        
        avg_result.append([n_split, evaluated[0], evaluated[1]])
        


``` Click the button to reveal!
y_pred = model.predict(xtest)
ytest = np.argmax(ytest, axis=1)
ypred = np.argmax(y_pred, axis=1)
        
print("Accuracy: {}".format(accuracy_score(ytest, ypred)))
print("Precision: {}".format(precision_score(ytest, ypred, average='micro')))
print("Recall: {}".format(recall_score(ytest, ypred, average='micro')))
print("F1-Score: {}".format(f1_score(ytest, ypred, average='micro')))

```

In [8]:
%%time
for n in range(2, 11):
    kfold(n, X, y)

127/127 [==============================] - 1s 8ms/step - loss: 2.8768 - accuracy: 0.6719
Model evaluation  [2.876765012741089, 0.6718518733978271]
127/127 [==============================] - 1s 9ms/step - loss: 2.8873 - accuracy: 0.6852
Model evaluation  [2.8872811794281006, 0.6851851940155029]
85/85 [==============================] - 1s 9ms/step - loss: 2.1234 - accuracy: 0.7300
Model evaluation  [2.1233997344970703, 0.7300000190734863]
85/85 [==============================] - 1s 9ms/step - loss: 2.4010 - accuracy: 0.7381
Model evaluation  [2.401017189025879, 0.7381481528282166]
85/85 [==============================] - 1s 9ms/step - loss: 2.0670 - accuracy: 0.7530
Model evaluation  [2.067049980163574, 0.7529629468917847]
64/64 [==============================] - 1s 9ms/step - loss: 2.0504 - accuracy: 0.7170
Model evaluation  [2.0503976345062256, 0.7170370221138]
64/64 [==============================] - 1s 9ms/step - loss: 1.8016 - accuracy: 0.7630
Model evaluation  [1.8016307353973389, 

In [9]:
avg_result

[[2, 2.876765012741089, 0.6718518733978271],
 [2, 2.8872811794281006, 0.6851851940155029],
 [3, 2.1233997344970703, 0.7300000190734863],
 [3, 2.401017189025879, 0.7381481528282166],
 [3, 2.067049980163574, 0.7529629468917847],
 [4, 2.0503976345062256, 0.7170370221138],
 [4, 1.8016307353973389, 0.7629629373550415],
 [4, 1.9742584228515625, 0.7432098984718323],
 [4, 2.05108642578125, 0.7590123414993286],
 [5, 2.091592311859131, 0.7456790208816528],
 [5, 1.5562652349472046, 0.7771604657173157],
 [5, 1.8911265134811401, 0.7611111402511597],
 [5, 1.8355718851089478, 0.7666666507720947],
 [5, 1.8389885425567627, 0.7870370149612427],
 [6, 1.5681517124176025, 0.7518518567085266],
 [6, 1.5097110271453857, 0.7985185384750366],
 [6, 1.5440239906311035, 0.8007407188415527],
 [6, 1.9054971933364868, 0.7555555701255798],
 [6, 1.5735678672790527, 0.787407398223877],
 [6, 1.752150058746338, 0.770370364189148],
 [7, 1.7943357229232788, 0.7314335107803345],
 [7, 1.7967779636383057, 0.7744165658950806],


In [15]:
df = pd.DataFrame(avg_result, columns=["K", "Loss", "Accuracy"])

In [16]:
df.to_csv('kfold_validation.csv', index=False)

In [17]:
df

,K,Loss,Accuracy
0,2,2.876765,0.671852
1,2,2.887281,0.685185
2,3,2.123400,0.730000
3,3,2.401017,0.738148
4,3,2.067050,0.752963
5,4,2.050398,0.717037
6,4,1.801631,0.762963
7,4,1.974258,0.743210
8,4,2.051086,0.759012
9,5,2.091592,0.745679
